# Tugas Clustering 

##### Ilham Muhammad Misbahuddin
##### 05111540000088

## Kebutuhan :
1. Operating System : Kali Linux 2019.1
2. Apache Spark 2.3.3
3. Scala 2.12.8
4. Python 3.7.3rc1
5. PySpark 2.4.0
6. Findspark 1.3.0
7. Jupyter 4.4.0

## Deskripsi Dataset
* Nama Dataset : [Online Retail](https://www.kaggle.com/puneetbhaya/online-retail)

<table>
    <thead>
        <tr>
            <th>Sumber Data</th>
            <th>Jumlah Baris</th>
            <th>Jumlah Colom</th>
            <th>Ukuran</th>
            <th>Format File</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Online Retail.xlsx</td>
            <td>541909</td>
            <td>8</td>
            <td>23,7 MB</td>
            <td>XLSX</td>
        </tr>
    </tbody>
</table>
    


## Inisialisasi Apache Spark

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder.appName("Big Data Frequent Itemsets").getOrCreate()

print(spark)

## Load Dataset

In [2]:
# Load excel
import pandas as pd
from pyspark.sql.types import *

df_excel = pd.read_excel("/root/Lecture/BIGDATA/datasets/Online Retail.xlsx")

header = StructType([ StructField("InvoiceNo", StringType(), True)\
                       ,StructField("StockCode", StringType(), True)\
                       ,StructField("Description", StringType(), True)\
                       ,StructField("Quantity", IntegerType(), True)\
                       ,StructField("InvoiceDate", StringType(), True)\
                       ,StructField("UnitPrice", DoubleType(), True)\
                       ,StructField("CustomerID", StringType(), True)\
                       ,StructField("Country", StringType(), True)])

df = spark.createDataFrame(df_excel, schema=header)
type(df)

pyspark.sql.dataframe.DataFrame

In [3]:
# Count dataset Online Retail
df.count()

541909

In [4]:
# Show dataset Online Retail
df.show()

+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|         InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|java.util.Gregori...|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|java.util.Gregori...|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|java.util.Gregori...|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|java.util.Gregori...|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|java.util.Gregori...|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|java.util.Gregori...|     7.65|   17850.0|United Kingdom|
|   536365|    2173

In [5]:
# Drop some column
dropped_column = ['Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'CustomerID']
df = df.drop(*dropped_column)

# Show dataset after some column dropped
df.show()

+---------+---------+--------------------+
|InvoiceNo|StockCode|         Description|
+---------+---------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|
|   536365|    71053| WHITE METAL LANTERN|
|   536365|   84406B|CREAM CUPID HEART...|
|   536365|   84029G|KNITTED UNION FLA...|
|   536365|   84029E|RED WOOLLY HOTTIE...|
|   536365|    22752|SET 7 BABUSHKA NE...|
|   536365|    21730|GLASS STAR FROSTE...|
|   536366|    22633|HAND WARMER UNION...|
|   536366|    22632|HAND WARMER RED P...|
|   536367|    84879|ASSORTED COLOUR B...|
|   536367|    22745|POPPY'S PLAYHOUSE...|
|   536367|    22748|POPPY'S PLAYHOUSE...|
|   536367|    22749|FELTCRAFT PRINCES...|
|   536367|    22310|IVORY KNITTED MUG...|
|   536367|    84969|BOX OF 6 ASSORTED...|
|   536367|    22623|BOX OF VINTAGE JI...|
|   536367|    22622|BOX OF VINTAGE AL...|
|   536367|    21754|HOME BUILDING BLO...|
|   536367|    21755|LOVE BUILDING BLO...|
|   536367|    21777|RECIPE BOX WITH M...|
+---------+

In [6]:
# Group dataset 
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

distinct_items= udf(lambda row: list(set(row)), ArrayType(StringType()))
df_group = df.groupby("InvoiceNo").agg(F.collect_set("StockCode"))
df_group = df_group.withColumn("distinct_items", distinct_items("collect_set(StockCode)"))

# Show groupped dataset
df_group.show()

+---------+----------------------+--------------------+
|InvoiceNo|collect_set(StockCode)|      distinct_items|
+---------+----------------------+--------------------+
|   536596|  [22900, 22114, 84...|[21624, 22114, 21...|
|   536938|  [22112, 21931, 84...|[84997A, 21479, 8...|
|   537252|               [22197]|             [22197]|
|   537691|  [22505, 46000R, 2...|[22505, 22666, 84...|
|   538041|               [22145]|             [22145]|
|   538184|  [22561, 22147, 21...|[22560, 22561, 21...|
|   538517|  [22749, 21212, 22...|[22562, 22548, 22...|
|   538879|  [21212, 22759, 22...|[22593, 22453, 21...|
|   539275|  [22083, 22150, 22...|[21914, 21915, 22...|
|   539630|  [22111, 22971, 22...|[21484, 21485, 22...|
|   540499|  [22697, 22796, 21...|[22625, 22624, 22...|
|   540540|  [22111, 22834, 22...|[22443, 22442, 22...|
|   540976|  [22413, 21212, 22...|[21908, 22115, 22...|
|   541432|  [22113, 22457, 21...|[21485, 22113, 22...|
|   541518|  [21212, 22432, 22...|[22504, 22378,

In [7]:
# Select column to be saved to check the items
df_check = df.selectExpr(['InvoiceNo', 'StockCode', 'Description'])

df_check.show()

+---------+---------+--------------------+
|InvoiceNo|StockCode|         Description|
+---------+---------+--------------------+
|   536365|   85123A|WHITE HANGING HEA...|
|   536365|    71053| WHITE METAL LANTERN|
|   536365|   84406B|CREAM CUPID HEART...|
|   536365|   84029G|KNITTED UNION FLA...|
|   536365|   84029E|RED WOOLLY HOTTIE...|
|   536365|    22752|SET 7 BABUSHKA NE...|
|   536365|    21730|GLASS STAR FROSTE...|
|   536366|    22633|HAND WARMER UNION...|
|   536366|    22632|HAND WARMER RED P...|
|   536367|    84879|ASSORTED COLOUR B...|
|   536367|    22745|POPPY'S PLAYHOUSE...|
|   536367|    22748|POPPY'S PLAYHOUSE...|
|   536367|    22749|FELTCRAFT PRINCES...|
|   536367|    22310|IVORY KNITTED MUG...|
|   536367|    84969|BOX OF 6 ASSORTED...|
|   536367|    22623|BOX OF VINTAGE JI...|
|   536367|    22622|BOX OF VINTAGE AL...|
|   536367|    21754|HOME BUILDING BLO...|
|   536367|    21755|LOVE BUILDING BLO...|
|   536367|    21777|RECIPE BOX WITH M...|
+---------+

## Prediksi Frequent Itemsets

In [8]:
# Frequent itemsets with Minimum Support = 0.05 and Minimum Confidence = 0.8
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="collect_set(StockCode)", minSupport=0.05, minConfidence=0.8)
model = fpGrowth.fit(df_group)
model.freqItemsets.show()
model.freqItemsets.count()

+--------+----+
|   items|freq|
+--------+----+
|[85123A]|2246|
| [22423]|2172|
|[85099B]|2135|
| [47566]|1706|
| [20725]|1608|
| [84879]|1468|
| [22720]|1462|
| [22197]|1442|
| [21212]|1334|
| [22383]|1306|
| [20727]|1295|
+--------+----+



11

In [9]:
# Association rule with Minimum Support = 0.05 and Minimum Confidence = 0.8
model.associationRules.show()

+----------+----------+----------+
|antecedent|consequent|confidence|
+----------+----------+----------+
+----------+----------+----------+



In [10]:
# Item prediction with Minimum Support = 0.05 and Minimum Confidence = 0.8
model.transform(df_group).show()

+---------+----------------------+--------------------+----------+
|InvoiceNo|collect_set(StockCode)|      distinct_items|prediction|
+---------+----------------------+--------------------+----------+
|   536596|  [22900, 22114, 84...|[21624, 22114, 21...|        []|
|   536938|  [22112, 21931, 84...|[84997A, 21479, 8...|        []|
|   537252|               [22197]|             [22197]|        []|
|   537691|  [22505, 46000R, 2...|[22505, 22666, 84...|        []|
|   538041|               [22145]|             [22145]|        []|
|   538184|  [22561, 22147, 21...|[22560, 22561, 21...|        []|
|   538517|  [22749, 21212, 22...|[22562, 22548, 22...|        []|
|   538879|  [21212, 22759, 22...|[22593, 22453, 21...|        []|
|   539275|  [22083, 22150, 22...|[21914, 21915, 22...|        []|
|   539630|  [22111, 22971, 22...|[21484, 21485, 22...|        []|
|   540499|  [22697, 22796, 21...|[22625, 22624, 22...|        []|
|   540540|  [22111, 22834, 22...|[22443, 22442, 22...|       

In [11]:
# Frequent itemsets with Minimum Support = 0.03 and Minimum Confidence = 0.7
fpGrowth = FPGrowth(itemsCol="collect_set(StockCode)", minSupport=0.03, minConfidence=0.7)
model2 = fpGrowth.fit(df_group)
model2.freqItemsets.show()
model2.freqItemsets.count()

+---------------+----+
|          items|freq|
+---------------+----+
|       [85123A]|2246|
|        [22423]|2172|
|       [85099B]|2135|
|        [47566]|1706|
|        [20725]|1608|
|        [84879]|1468|
|        [22720]|1462|
|        [22197]|1442|
|        [21212]|1334|
|        [22383]|1306|
|        [20727]|1295|
|        [22457]|1266|
|         [POST]|1254|
|        [23203]|1249|
|        [22386]|1231|
|[22386, 85099B]| 833|
|        [22960]|1220|
|        [22469]|1214|
|        [21931]|1201|
|        [22411]|1187|
+---------------+----+
only showing top 20 rows



80

In [12]:
# Association rule with Minimum Support = 0.03 and Minimum Confidence = 0.7
model2.associationRules.show()

+----------+----------+------------------+
|antecedent|consequent|        confidence|
+----------+----------+------------------+
|   [22699]|   [22697]|               0.7|
|   [22697]|   [22699]|0.7417218543046358|
+----------+----------+------------------+



In [13]:
# Item prediction with Minimum Support = 0.03 and Minimum Confidence = 0.7
model2.transform(df_group).show()

+---------+----------------------+--------------------+----------+
|InvoiceNo|collect_set(StockCode)|      distinct_items|prediction|
+---------+----------------------+--------------------+----------+
|   536596|  [22900, 22114, 84...|[21624, 22114, 21...|        []|
|   536938|  [22112, 21931, 84...|[84997A, 21479, 8...|        []|
|   537252|               [22197]|             [22197]|        []|
|   537691|  [22505, 46000R, 2...|[22505, 22666, 84...|        []|
|   538041|               [22145]|             [22145]|        []|
|   538184|  [22561, 22147, 21...|[22560, 22561, 21...|        []|
|   538517|  [22749, 21212, 22...|[22562, 22548, 22...|        []|
|   538879|  [21212, 22759, 22...|[22593, 22453, 21...|        []|
|   539275|  [22083, 22150, 22...|[21914, 21915, 22...|        []|
|   539630|  [22111, 22971, 22...|[21484, 21485, 22...|        []|
|   540499|  [22697, 22796, 21...|[22625, 22624, 22...|        []|
|   540540|  [22111, 22834, 22...|[22443, 22442, 22...|       

In [14]:
# Frequent itemsets with Minimum Support = 0.01 and Minimum Confidence = 0.6
fpGrowth = FPGrowth(itemsCol="collect_set(StockCode)", minSupport=0.01, minConfidence=0.6)
model3 = fpGrowth.fit(df_group)
model3.freqItemsets.show()
model3.freqItemsets.count()

+----------------+----+
|           items|freq|
+----------------+----+
|         [22633]| 487|
|         [23236]| 344|
|        [85123A]|2246|
|         [22423]|2172|
| [22423, 85123A]| 355|
|         [22667]| 486|
|         [22579]| 343|
|  [22579, 22578]| 282|
|        [85099B]|2135|
| [85099B, 22423]| 288|
|[85099B, 85123A]| 404|
|         [22620]| 486|
|        [84536A]| 342|
|         [71053]| 342|
|         [47566]|1706|
| [47566, 85099B]| 332|
|  [47566, 22423]| 398|
| [47566, 85123A]| 391|
|         [85150]| 483|
|         [20725]|1608|
+----------------+----+
only showing top 20 rows



1087

In [ ]:
# Association rule with Minimum Support = 0.01 and Minimum Confidence = 0.6
model3.associationRules.show()

In [ ]:
# Item prediction with Minimum Support = 0.01 and Minimum Confidence = 0.6
model3.transform(df_group).show()

In [ ]:
model3.transform(df_group).printSchema()

In [ ]:
# Get 1 example of items and put it into Data Frame
df_compare=spark.createDataFrame([('0',['22697'])],['id','collect_set(StockCode)'])

df_compare.show()

In [ ]:
# Find the closest prediction for item 22697
model3.transform(df_compare).show()

In [ ]:
df_check.createOrReplaceTempView("Online_Retail")

In [ ]:
# Find the item with StockCode of '22697'
query1 = spark.sql("SELECT DISTINCT Description FROM Online_Retail WHERE StockCode = '22697'")
query1.show()

In [ ]:
# Find the item with StockCode of '22699'
query2 = spark.sql("SELECT DISTINCT Description FROM Online_Retail WHERE StockCode = '22699'")
query2.show()

In [ ]:
# Find the item with StockCode of '22698'
query3 = spark.sql("SELECT DISTINCT Description FROM Online_Retail WHERE StockCode = '22698'")
query3.show()